# Introduction to Snakemake
---

<strong>Public Health Bioinormatics</strong> (Hsiao's lab) 

The materials are available on Github at https://github.com/Public-Health-Bioinformatics/snakemake_demo

The Snakemake workflow management system is a tool to create reproducible and scalable data analyses. It orchestrates and keeps track of all the different steps of workflows that have been run. It becomes more and more popular because: 

- Snakemake is written using Python, but supports bash and R code as well.
- It’s free, open-source, and conda-installable
- Snakemake works cross-platform (Windows, MacOS, Linux) and cloud.

Like other workflow management systems, Snakemake allows you to:

- Keep a record of how your scripts are used and what their input dependencies are
- Run multiple steps in sequence, parallelising where possible
- Automatically detect if something changes and then reprocess data if needed


## 1. installation
Snakemake is available on PyPi as well as through Bioconda and also from source code. However, the recommended way of installation is using conda. 

### Install conda
If you haven't had conda yet, you can install miniconda like this:

In [ ]:
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

In [ ]:
sh Miniconda3-latest-Linux-x86_64.sh

### Install Snakemake through conda

In [ ]:
conda create -c bioconda -c conda-forge -n snakemake_demo snakemake-minimal -y

Or you can mount your existing conda environment, then install like this:

In [ ]:
conda install snakemake-minimal

### Check if snakemake is working

In [ ]:
conda activate snakemake_demo

In [ ]:
snakemake --version

### To deactivate an active environment:

In [ ]:
conda deactivate

## 2. Basic rules of snakemake

A Snakemake workflow defines a data analysis in terms of rules, that are listed in so-called Snakefiles. Most importantly, a rule can consist of a name, input files, output files, and a shell command to generate the output from the input.



In [ ]:
### Example 1.1

```
rule copy:
    input:
        "A_input.txt"
    output:
        "A_copy.txt"
    shell:
        "cp {input} {output}"
```

Let's make a file named Snakefile and put the above rule into the file. Tips: I have prepared the files and you can download from GitHub directly.To run the examples, you need to make sure to have Snakemake installed on your system either locally or in a snakemake specific conda environment:


In [16]:
git clone --recursive https://github.com/Public-Health-Bioinformatics/snakemake_demo.git

Cloning into 'github.com'...
fatal: repository 'https://github.com/' not found
(snakemake_demo) 

: 1

In [1]:
cd snakemake_demo/demo_example/example_1/

bash: cd: snakemake_demo/demo_example/example_1/: No such file or directory
/home/dj/snakemake_demo_work/snakemake_demo


In [2]:
pwd

/home/dj/snakemake_demo_work/snakemake_demo


Let's do a dry run first.

In [ ]:
snakemake -n -p

Let's do a actual run

In [ ]:
snakemake -j 1

### Example 1.2


In [ ]:
After we copy the file, how can we change all the letters to upcase?

In [ ]:
```
rule copy:
    input:
        "A_input.txt"
    output:
        "A_copy.txt"
    shell:
        "cp {input} {output}"

rule uppercase:
    input:
        "A_copy.txt"
    output:
        "A_output.txt"
    shell:
        "dd if={input} of={output} conv=ucase"

```

In [20]:
snakemake -j 1 -s example_snakefile_1.1 A_output.txt

Building DAG of jobs...
Nothing to be done.
Complete log: /home/dj/snakemake_demo/demo/example_1/.snakemake/log/2021-01-20T175530.945759.snakemake.log
(snakemake_demo) 

: 1

#### Generate DAG graph
Snakemake is able to create a directed acyclic graph (DAG) that represents a plan of rule executions. The nodes of the DAG are jobs, a directed edge means the dependency.

First, make sure graphviz is installed. If not, you can install it like this:


In [ ]:
conda install graphviz -f

Then you can generate the DAG graph:

In [ ]:
snakemake -s example1.2_snakefile --dag  | dot -Tpng > dag.png

### Example 1.3

What should we do if we have multipe files to process like A_input.txt, B_input.txt, C_input.txt ...

In [ ]:
```
rule copy:
    input:
        "{file}_input.txt"
    output:
        "{file}_copy.txt"
    shell:
        "cp {input} {output}"

rule uppercase:
    input:
        "{file}_copy.txt"
    output:
        "{file}_output.txt"
    shell:
        "dd if={input} of={output} conv=ucase"
        
```

First, let's look at the DAG graph

In [ ]:
snakemake -s example1.3_snakefile --dag {A,B,C}_output.txt | dot -Tpng > dag.png

In [ ]:
We can also do a dry run:

In [ ]:
snakemake -s example1.3_snakefile -n -p  {A,B,C}_output.txt

### Example 1.4

In [ ]:
Another way of doing above jobs

In [ ]:
```
datasets=["A","B","C"]

rule all:
    input: "all.txt"
        

rule copy:
    input:
        "{file}_input.txt"
    output:
        "{file}_copy.txt"
    shell:
        "cp {input} {output}"

rule uppercase:
    input:
        "{file}_copy.txt"
    output:
        "{file}_output.txt"
    shell:
        "dd if={input} of={output} conv=ucase"
        
rule combine:
    input:
        expand("{id}_output.txt", id=datasets)
    output:
        "all.txt"
    shell:
        "cat {input} > {output}"
        
```

In [ ]:
snakemake -s example1.4_snakefile --dag | dot -Tpng > dag.png

In [ ]:
snakemake -s example1.4_snakefile -np

We can also put multiple dataset to expand. For example:

expand(["{dataset}.{ext}", "{dataset}.{ext}"], dataset=[A1,A2], ext=[png,jpg])

will lead to

["A1.png", "A1.jpg", "A2.png","A2.jpg"]


We can generate snakemake report to get details about the running status. Before this, please make sure Jinja2, networkx, pygments and pygraphvi is installed. If not, please install it using conda.


In [ ]:
conda install networkx pygraphvi Jinja2 pygments -y

In [ ]:
snakemake -s example1.4_snakefile --report report.html

### 3. More examples for data analysis

### Example 2.1

#### Use fastqc generate QC report (example_2.1)


In [ ]:
cd snakemake_demo/demo_example/example_2

In [ ]:
```
SAMPLES = ["Sample1", "Sample2", "Sample3"]


rule fastqc: 
    input: 
        expand("data/{sample}.fastq", sample=SAMPLES)
    output:
        expand("data/{sample}_fastqc.html", sample=SAMPLES)
    conda: 
        "env.yml"
    params: 
        threads = "1"
    shell:
        "fastqc -t {params.threads} {input}"
        
```


In [ ]:
snakemake -s example2.1_snakefile --dag | dot -Tpng > dag.png

In [ ]:
snakemake -s example2.1_snakefile -np

#### Use multiqc to aggregate QC results (example_2.2)

In [ ]:
```
SAMPLES = ["Sample1", "Sample2", "Sample3"]

rule all:
    input:
        "multiqc_result/multiqc_report.html",

rule fastqc: 
    input: 
        expand("data/{sample}.fastq", sample=SAMPLES)
    output:
        expand("data/{sample}_fastqc.html", sample=SAMPLES)
    conda: 
        "env.yml"
    params: 
        threads = "3"
    shell:
        "fastqc -t {params.threads} {input}"

rule multiqc: 
    input: 
        expand("data/{sample}_fastqc.html", sample=SAMPLES)
    output:
        "multiqc_result/multiqc_report.html"
    conda: 
        "env.yml"
    shell:
        "multiqc data -o multiqc_result"
        
```

In [ ]:
snakemake -s example2.2_snakefile --dag | dot -Tpng > dag.png

In [ ]:
snakemake -s example2.2_snakefile -np

### Use glob_wildcards (example_2.3)

In [ ]:
```
SAMPLES, = glob_wildcards("data/{sample}.fastq")

rule all:
    input:
        "multiqc_result/multiqc_report.html",

rule fastqc: 
    input: 
        expand("data/{sample}.fastq", sample=SAMPLES)
    output:
        expand("data/{sample}_fastqc.html", sample=SAMPLES)
    conda: 
        "env.yml"
    params: 
        threads = "3"
    shell:
        "fastqc -t {params.threads} {input}"

rule multiqc: 
    input: 
        expand("data/{sample}_fastqc.html", sample=SAMPLES)
    output:
        "multiqc_result/multiqc_report.html"
    conda: 
        "env.yml"
    shell:
        "multiqc data -o multiqc_result"
```

In [ ]:
snakemake -s example2.3_snakefile --dag | dot -Tpng > dag.png

In [ ]:
snakemake -s example2.3_snakefile -np

## Resources

### 1. Snakemake
* Documentation: https://snakemake.readthedocs.io/
*  Wrappers: https://snakemake-wrappers.readthedocs.io/
* Snakemake-Workflows project: https://github.com/snakemake-workflows/docs
*  Snakemake-Profiles project: https://github.com/snakemake-profiles/doc


### 2. Workflows
* An incomplete list of 288 Computational Data Analysis Workflow Systems:
https://github.com/common-workflow-language/common-workflow-language/wiki/Existing-Workflow-systems
* A curated list of awesome pipeline toolkits:
https://github.com/pditommaso/awesome-pipeline